In [22]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt

def bandpass_filter(y, sr, low, high):
    sos = scipy.signal.butter(4, [low, high], btype='bandpass', fs=sr, output='sos')
    return scipy.signal.sosfilt(sos, y)

def estimate_bpm(y, sr, low=30, high=120, plot=False):
    # Ограничим анализ до 60 секунд
    y = y[:min(len(y), sr * 60)]

    # Применяем фильтрацию
    y_filtered = bandpass_filter(y, sr, low, high)

    # Считаем огибающую
    envelope = np.abs(y_filtered)
    envelope = scipy.signal.medfilt(envelope, kernel_size=31)

    # Порог: 95 перцентиль
    peak_thresh = np.percentile(envelope, 95)

    # Поиск пиков
    peaks, props = scipy.signal.find_peaks(
        envelope,
        distance=int(sr * 0.4),
        height=peak_thresh,
        width=int(sr * 0.05)
    )

    if len(peaks) < 2:
        return None, peaks

    # Расчёт BPM
    intervals = np.diff(peaks) / sr
    avg_interval = np.mean(intervals)
    bpm = 60 / avg_interval

    if plot and (40 <= bpm <= 180):
        times = np.arange(len(envelope)) / sr
        plt.figure(figsize=(12, 4))
        plt.plot(times, envelope)
        plt.plot(peaks / sr, envelope[peaks], 'rx')
        plt.title(f'Оцененная ЧСС = {bpm:.1f} BPM | Полоса: {low}-{high} Гц')
        plt.xlabel("Время (сек)")
        plt.tight_layout()
        plt.show()

    return bpm, peaks

def adaptive_heart_rate_from_wav(y, sr, plot=True):
    ranges = [
        (30, 120),
        (30, 100),
        (30, 80),
        (40, 100),
        (50, 150)
    ]
    for low, high in ranges:
        bpm, peaks = estimate_bpm(y, sr, low, high, plot=plot)
        if bpm is None:
            continue
        if 40 <= bpm <= 180:
            return bpm
        else: 
            return bpm
    return None


In [12]:
import IPython.display as ipd

In [13]:
ipd.Audio("../data/CDPD/training_data/50336_MV.wav")

In [ ]:
import librosa

y, sr = librosa.load("your_heart_sound.wav", sr=None)
bpm = adaptive_heart_rate_from_wav(y, sr, plot=True)

print(f"Частота сердцебиения: {bpm:.1f} BPM" if bpm else "Не удалось определить ЧСС")

In [23]:
wav_directory = "../data/CDPD/training_data"

# Анализ всех WAV-файлов в директории
for filename in os.listdir(wav_directory)[:10]:
    if filename.endswith(".wav"):
        filepath = os.path.join(wav_directory, filename)
        y, sr = librosa.load(filepath, sr=None)
        bpm = adaptive_heart_rate_from_wav(y, sr, plot=True)
        if bpm:
            print(f"{filename}: Оцененная ЧСС = {bpm:.1f} BPM")
        else:
            print(f"{filename}: Недостаточно данных для анализа.")


value -  (116160,)
value -  (116160,)
50336_MV.wav: Оцененная ЧСС = 21.6 BPM
value -  (125952,)
value -  (125952,)
value -  (125952,)
value -  (125952,)
value -  (125952,)
50048_TV.wav: Недостаточно данных для анализа.
value -  (41408,)
value -  (41408,)
value -  (41408,)
value -  (41408,)
value -  (41408,)
68347_TV.wav: Недостаточно данных для анализа.
value -  (121408,)
value -  (121408,)
value -  (121408,)
value -  (121408,)
value -  (121408,)
50619_TV.wav: Недостаточно данных для анализа.
value -  (101952,)
value -  (101952,)
value -  (101952,)
value -  (101952,)
value -  (101952,)
84960_MV.wav: Недостаточно данных для анализа.
